In [4]:
import os 
import pandas as pd
import numpy as np
from collections import Counter
from ast import literal_eval

In [2]:
DATA_DIR = os.getenv("DATA_DIR")
content_path = os.path.join(DATA_DIR, "preprocessed_content_store_en_june.csv.gz")
df = pd.read_csv(content_path, compression="gzip")

/Users/felisialoukou/.virtualenvs/k-graph/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['base_path', 'content_id', 'title', 'description', 'document_type',
       'orgs_id', 'orgs_title', 'sbs_details', 'pages_part_of_step_nav',
       'text', 'taxons', 'locale'],
      dtype='object')

In [5]:
columns = ['orgs_id', 'orgs_title', 'sbs_details', 'pages_part_of_step_nav','taxons']
for col in columns:
    df[col] = df[col].map(lambda x: literal_eval(x) if not isinstance(x,float) else np.nan)

In [6]:
df.head()

,base_path,content_id,title,description,document_type,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale
0,/aaib-reports/aaib-investigation-to-hawker-sea...,96eacfbe-0385-45ef-9289-8428dacad258,"AAIB investigation to Hawker Sea Fury T MK 20,...","Engine failure and landing gear collapse, RNAS...",aaib_report,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The aircraft was performing in a publ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
1,/aaib-reports/aaib-investigation-to-hph-glasfl...,1d697c99-b1d0-4855-b72d-a97d83a4fc91,"AAIB investigation to HPH Glasflugel 304 eS, G...",Front Electric Sustainer (FES) battery fire du...,aaib_report,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: During a normal touchdown following a...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
2,/aaib-reports/aaib-investigation-to-ikarus-c42...,5814334a-77d0-426e-8e78-ff2b05ea6322,"AAIB investigation to Ikarus C42 FB UK, G-IKUS\t",Aircraft crashed whilst avoiding a hedge when ...,aaib_report,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot was attempting to take off ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
3,/aaib-reports/aaib-investigation-to-ikarus-c42...,c8f31c76-eab0-4be6-95a5-5e6e7f32056c,"AAIB investigation to Ikarus C42 FB100, G-CEHG\t","Overturned on landing, Farm Strip, Hardwicke, ...",aaib_report,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: G-CEHG was landing on a private airst...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
4,/aaib-reports/aaib-investigation-to-ikarus-c42...,4cc0ae15-ad87-42ce-8a89-2bdb14e19e26,"AAIB investigation to Ikarus C42 FB100, G-ZAVI","Aircraft struck sheep on landing, Lundy Island...",aaib_report,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot reported that this was his ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en


### Create edgelist for `content_id` --> `step` (many to one)
#### Order of steps from `sbs_details`, node/page info (`base_path`, `content_id`)

In [52]:
example = df[~df.sbs_details.isna()].iloc[0]

In [53]:
example

base_path                                    /become-car-driving-instructor
content_id                             80bd32e4-9708-4214-bc0f-50358223bb09
title                                       Become a car driving instructor
description               You must become an approved driving instructor...
document_type                                              step_by_step_nav
orgs_id                   {'organisations': ['d39237a5-678b-4bb5-a372-eb...
orgs_title                {'organisations': ['Driver and Vehicle Standar...
sbs_details               {'title': 'Become a car driving instructor', '...
pages_part_of_step_nav    [{'analytics_identifier': None, 'api_path': '/...
text                                                                    NaN
taxons                    [{'title': 'Transport', 'content_id': 'a4038b2...
locale                                                                   en
sbs_base_path_cid         {'/apply-for-a-trainee-driving-instructor-lice...
Name: 11258,

In [15]:
example.sbs_details['steps']

[{'title': 'Check if you can become an instructor',
  'contents': [{'type': 'list',
    'contents': [{'text': "Check you're eligible to become a driving instructor",
      'href': '/become-a-driving-instructor'},
     {'text': 'Read about driving instructor responsibilities',
      'href': '/government/publications/whats-involved-in-being-a-driving-instructor'},
     {'text': 'Find out about the skills and knowledge you need',
      'href': '/government/publications/national-standard-for-driver-and-rider-training'}]}],
  'optional': False},
 {'title': 'Apply to become a driving instructor',
  'contents': [{'type': 'paragraph',
    'text': 'You must get a new Disclosure and Barring Service (DBS) check to start your application, even if you already have one.'},
   {'type': 'list',
    'contents': [{'text': 'Get a DBS check',
      'href': '/criminal-record-check-become-driving-instructor',
      'context': '£6'},
     {'text': 'Start your application',
      'href': '/apply-to-become-a-d

In [46]:
df['sbs_base_path_cid'] = df['pages_part_of_step_nav'].\
                                                map(lambda x:\
                                                {item['base_path']:item['content_id']\
                                                 for item in x} if not isinstance(x,float)\
                                                   else np.nan)

### Save out additional sbs stuff

In [49]:
sbs_content_path = os.path.join(DATA_DIR, "preprocessed_step_by_step_content_en_june.csv.gz")
df[~df.sbs_details.isna()].to_csv(sbs_content_path, index=False, compression="gzip")

In [ ]:
### edgelist

In [62]:
rowlist = []
sbs_details_col = df.columns.get_loc('sbs_details')
pages_nav_col = df.columns.get_loc('pages_part_of_step_nav')
cid_col = df.columns.get_loc('content_id')
base_path_col = df.columns.get_loc('base_path')
sbs_base_cid_col = df.columns.get_loc('sbs_base_path_cid')

for tup in df.itertuples(index=False):
    if not isinstance(tup[sbs_details_col],float):
        for i,step in enumerate(tup[sbs_details_col]['steps']):
            for content in step['contents']:
                if "contents" in content.keys():
                    for item in content['contents']:
#                         print(item)
                        if "href" in item.keys():
                            task_cid = np.nan
                            if not isinstance(tup[sbs_base_cid_col],float) and\
                                        item['href'] in tup[sbs_base_cid_col].keys():
                                task_cid = tup[sbs_base_cid_col][item['href']]
                            rowlist.append({
                                'content_id': tup[cid_col],
                                'base_path': tup[base_path_col],
                                'step_title': step['title'],
                                'step_number': i+1,
                                'task_base_path': item['href'],
                                'task_cid':task_cid})
df_steps = pd.DataFrame(rowlist)

In [63]:
df[(df.sbs_base_path_cid.isna()) & ~(df.sbs_details.isna())]

,base_path,content_id,title,description,document_type,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale,sbs_base_path_cid
291208,/how-to-claim-new-style-esa,a661e652-204c-4abf-a2ce-9f183aca85b9,How to claim 'new style' Employment and Suppor...,Find out if you're eligible for 'new style' Em...,step_by_step_nav,{'organisations': ['b548a09f-8b35-4104-89f4-f1...,{'organisations': ['Department for Work and Pe...,{'title': 'How to claim 'new style' Employment...,NaN,NaN,"[{'title': 'Welfare', 'content_id': 'dded88e2-...",en,NaN
291209,/how-to-claim-new-style-jsa,e7b3ea75-05d5-4341-b27a-be3b767b1e3f,How to claim 'new style' Jobseeker's Allowance...,Find out if you're eligible for 'new style' Jo...,step_by_step_nav,{'organisations': ['b548a09f-8b35-4104-89f4-f1...,{'organisations': ['Department for Work and Pe...,{'title': 'How to claim 'new style' Jobseeker'...,NaN,NaN,"[{'title': 'Welfare', 'content_id': 'dded88e2-...",en,NaN
295372,/visit-uk-private-medical-treatment,8a9b3663-2117-470a-9400-9144b76c71b1,Visit the UK for private medical treatment,Come to the UK for private medical treatment -...,step_by_step_nav,{'organisations': ['04148522-b0c1-4137-b687-5f...,{'organisations': ['UK Visas and Immigration']...,{'title': 'Visit the UK for private medical tr...,NaN,NaN,"[{'title': 'Entering and staying in the UK', '...",en,NaN
295418,/visit-uk-research,b1f1b767-776f-43ac-af4b-2bbbc39e3ad2,Visit the UK to do research,What to do if you're visiting the UK to do res...,step_by_step_nav,{'organisations': ['04148522-b0c1-4137-b687-5f...,{'organisations': ['UK Visas and Immigration']...,"{'title': 'Visit the UK to do research', 'intr...",NaN,NaN,"[{'title': 'Entering and staying in the UK', '...",en,NaN


In [65]:
df_steps[df_steps.task_cid.isna()]

,base_path,content_id,step_number,step_title,task_base_path,task_cid
6,/become-car-driving-instructor,80bd32e4-9708-4214-bc0f-50358223bb09,4,Take the theory test (ADI part 1),/adi-part-1-test/revision-practice,NaN
7,/become-car-driving-instructor,80bd32e4-9708-4214-bc0f-50358223bb09,4,Take the theory test (ADI part 1),/adi-part-1-test/what-to-take,NaN
10,/become-car-driving-instructor,80bd32e4-9708-4214-bc0f-50358223bb09,5,Take the driving ability test (ADI part 2),/adi-part-2-test/what-happens-during-test,NaN
11,/become-car-driving-instructor,80bd32e4-9708-4214-bc0f-50358223bb09,5,Take the driving ability test (ADI part 2),/adi-part-2-test/what-to-take,NaN
16,/become-car-driving-instructor,80bd32e4-9708-4214-bc0f-50358223bb09,7,Take the instructional ability test (ADI part 3),/adi-part-3-test/what-happens-during-test,NaN
17,/become-car-driving-instructor,80bd32e4-9708-4214-bc0f-50358223bb09,7,Take the instructional ability test (ADI part 3),/adi-part-3-test/what-to-take,NaN
26,/bring-your-pet-to-uk,9c8e6e6d-29f7-41d3-bc31-efcf20ee0d69,1,Check if you can bring your pet,/take-pet-abroad/approved-routes,NaN
27,/bring-your-pet-to-uk,9c8e6e6d-29f7-41d3-bc31-efcf20ee0d69,1,Check if you can bring your pet,/take-pet-abroad/guide-dogs,NaN
28,/bring-your-pet-to-uk,9c8e6e6d-29f7-41d3-bc31-efcf20ee0d69,2,Get your pet microchipped,/take-pet-abroad/microchip,NaN
29,/bring-your-pet-to-uk,9c8e6e6d-29f7-41d3-bc31-efcf20ee0d69,3,Get your pet vaccinated against rabies,/take-pet-abroad/rabies-vaccination-boosters-a...,NaN
